# Detect Damage Parts from an Image


In [4]:
from ultralytics import YOLO
import cv2

model = YOLO(r"models\best.pt") 

image_path = r"samples\images\img1.png"
results = model(image_path)

for result in results:
    result.show()


image 1/1 c:\Users\Hp\Desktop\Car damage detection\samples\images\img1.png: 608x640 1 Bumper, 85.3ms
Speed: 3.4ms preprocess, 85.3ms inference, 0.8ms postprocess per image at shape (1, 3, 608, 640)


# Detect Damage Parts from a Video (Q for exit the video )

In [5]:
from ultralytics import YOLO
import cv2

model = YOLO(r"models\best.pt") 

video_path = r"samples\Video.mp4"
cap = cv2.VideoCapture(video_path)  

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS)) 

out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

display_width = 1200  
display_height = 750

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break 
    
    results = model(frame)
    
    for result in results:
        annotated_frame = result.plot()
    
    annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)

    resized_frame = cv2.resize(annotated_frame, (display_width, display_height))

    cv2.imshow("YOLO Video Detection", resized_frame)

    out.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()  
cv2.destroyAllWindows()



0: 384x640 1 Bonnet, 81.0ms
Speed: 2.5ms preprocess, 81.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bonnet, 45.1ms
Speed: 3.1ms preprocess, 45.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bonnet, 43.5ms
Speed: 2.4ms preprocess, 43.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bonnet, 47.5ms
Speed: 2.8ms preprocess, 47.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bonnet, 1 Door, 47.4ms
Speed: 2.5ms preprocess, 47.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bonnet, 1 Door, 46.7ms
Speed: 2.7ms preprocess, 46.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bonnet, 41.4ms
Speed: 1.8ms preprocess, 41.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bonnet, 1 Light, 38.6ms
Speed: 1.7ms preprocess, 38.6ms inference, 0.7ms postprocess per

# Image-Based Damage Detection and Classification

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

yolo_model = YOLO(r"models\best.pt")

classifier_model = load_model("models\damage_classifier.h5")
classifier_labels = ["Repair", "Replace"]

image_path = r"samples\images\img2.jpeg"

image = cv2.imread(image_path)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = yolo_model(rgb_image)[0]

for box in results.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cls_id = int(box.cls[0])
    part_label = yolo_model.names[cls_id]

    cropped = rgb_image[y1:y2, x1:x2]
    if cropped.size == 0:
        continue

    resized = cv2.resize(cropped, (224, 224))
    input_array = img_to_array(resized) / 255.0
    input_array = np.expand_dims(input_array, axis=0)

    prediction = classifier_model.predict(input_array, verbose=0)
    label_idx = np.argmax(prediction)
    status_label = classifier_labels[label_idx]
    status_conf = float(np.max(prediction))

    print(f"[INFO] Part: {part_label} | Status: {status_label} | Confidence: {status_conf * 100:.2f}%")

    color = (0, 255, 0) if status_label == "Repair" else (0, 0, 255)

    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    label_text = f"{part_label} | {status_label} ({status_conf:.2f})"

    text_size, _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
    text_width, text_height = text_size

    label_y = y1 - 10 if y1 - 10 > text_height else y1 + text_height + 10

    cv2.rectangle(image, (x1, label_y - text_height - 5), (x1 + text_width + 5, label_y + 5), color, -1)
    cv2.putText(image, label_text, (x1 + 2, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

resized = cv2.resize(image, (1200, 750))
cv2.imshow("Full Image with Damage Prediction", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 480x640 1 Door, 76.0ms
Speed: 3.3ms preprocess, 76.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
[INFO] Part: Door | Status: Repair | Confidence: 50.06%


# Video-Based Damage Detection and Classification

In [6]:
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

yolo_model = YOLO(r"models\best.pt")

classifier_model = load_model("models\damage_classifier.h5")
classifier_labels = ["Repair", "Replace"]

video_path = r"samples\Video.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = yolo_model(rgb_frame)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls_id = int(box.cls[0])    
        part_label = yolo_model.names[cls_id]

        cropped = rgb_frame[y1:y2, x1:x2]
        if cropped.size == 0:
            continue

        resized = cv2.resize(cropped, (224, 224))
        input_array = img_to_array(resized) / 255.0
        input_array = np.expand_dims(input_array, axis=0)

        prediction = classifier_model.predict(input_array, verbose=0)
        label_idx = np.argmax(prediction)
        status_label = classifier_labels[label_idx]
        status_conf = float(np.max(prediction))
        
        
        print(f"[INFO] Part: {part_label} | Status: {status_label} | Confidence: {status_conf * 100:.2f}%")

        color = (0, 255, 0) if status_label == "Repair" else (0, 0, 255)
        label_text = f"{part_label} | {status_label} ({status_conf:.2f})"

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        text_size, _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        text_width, text_height = text_size
        label_y = y1 - 10 if y1 - 10 > text_height else y1 + text_height + 10

        cv2.rectangle(frame, (x1, label_y - text_height - 5), (x1 + text_width + 5, label_y + 5), color, -1)
        cv2.putText(frame, label_text, (x1 + 2, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    output = cv2.resize(frame, (1200, 750))
    cv2.imshow("Video Damage Detection", output)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 Bonnet, 1 Bumper, 44.5ms
Speed: 2.1ms preprocess, 44.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Part: Bumper | Status: Replace | Confidence: 98.39%
[INFO] Part: Bonnet | Status: Replace | Confidence: 99.73%

0: 384x640 1 Bonnet, 1 Bumper, 42.2ms
Speed: 2.5ms preprocess, 42.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Part: Bonnet | Status: Replace | Confidence: 99.65%
[INFO] Part: Bumper | Status: Replace | Confidence: 99.17%

0: 384x640 1 Bonnet, 1 Bumper, 42.2ms
Speed: 1.5ms preprocess, 42.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Part: Bonnet | Status: Replace | Confidence: 99.71%
[INFO] Part: Bumper | Status: Replace | Confidence: 98.64%

0: 384x640 1 Bonnet, 1 Bumper, 37.6ms
Speed: 1.6ms preprocess, 37.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
[INFO] Part: Bumper | Status: Replace | Confidence: 98.91%
[INFO] Part: Bonnet | Status: Replace | Con